# Download Data

In [1]:
import subprocess
import os

download_savepath = './data/downloads'
os.makedirs(download_savepath, exist_ok=True)

if not os.path.exists('%s/buen_ad_sc.h5ad'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/buen_ad_sc.h5ad'%download_savepath, shell=True)

if not os.path.exists('%s/buen_model_sc.h5'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/buen_model_sc.h5'%download_savepath, shell=True)

if not os.path.exists('%s/pbmc_ad.h5ad'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/pbmc_ad.h5ad'%download_savepath, shell=True)

if not os.path.exists('%s/pbmc_model.h5'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/pbmc_model.h5'%download_savepath, shell=True)

# download motif injection fasta files for CISBP-1.0 motifs and unzip:
if not os.path.exists('%s/Homo_sapiens_motif_fasta.tar.gz'%download_savepath):
    subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/Homo_sapiens_motif_fasta.tar.gz'%download_savepath, shell=True)
subprocess.run('tar -xzf %s/Homo_sapiens_motif_fasta.tar.gz -C %s/'%(download_savepath, download_savepath), shell=True)

CompletedProcess(args='tar -xzf ./data/downloads/Homo_sapiens_motif_fasta.tar.gz -C ./data/downloads/', returncode=0)

# Create AnnData

In [2]:
! pip install biopython -- quiet
! pip install scanpy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement quiet (from versions: none)
ERROR: No matching distribution found for quiet
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00


In [3]:
! git clone https://github.com/calico/scBasset.git
! pip install -e scBasset --quiet

Cloning into 'scBasset'...
remote: Enumerating objects: 607, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 607 (delta 65), reused 90 (delta 37), pack-reused 485
Receiving objects: 100% (607/607), 93.66 MiB | 31.80 MiB/s, done.
Resolving deltas: 100% (266/266), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.3/375.3 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.

Then Restart Runtime

In [26]:
import numpy as np
import pandas as pd
import h5py
import scipy
import scanpy as sc
import anndata
from scbasset.utils import *

from Bio import SeqIO
from collections import Counter

# plotting functions
import seaborn as sns
import matplotlib.pyplot as plt

import os

read example from 10x multiome output

In [2]:
data_path = '/content/scBasset/data/multiome_pbmc/'

h5_file = data_path + 'pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.h5'
bed_file = data_path + 'pbmc_granulocyte_sorted_3k_atac_peaks.bed'

In [3]:
ad = sc.read_10x_h5(h5_file, gex_only=False)
peak = pd.read_csv(bed_file, sep='\t', names=['chr','start','end'])

/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
ad.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
KI270713.1:20252-22762,KI270713.1:20252-22762,Peaks,GRCh38
KI270713.1:26456-28921,KI270713.1:26456-28921,Peaks,GRCh38
KI270713.1:29478-30798,KI270713.1:29478-30798,Peaks,GRCh38
KI270713.1:31532-33156,KI270713.1:31532-33156,Peaks,GRCh38


In [5]:
ad

AnnData object with n_obs × n_vars = 2714 × 193208
    var: 'gene_ids', 'feature_types', 'genome'

In [6]:
peak

,chr,start,end
0,chr1,9986,10403
1,chr1,180690,181695
2,chr1,191335,192015
3,chr1,267775,268224
4,chr1,271144,271510
...,...,...,...
156602,KI270713.1,20252,22762
156603,KI270713.1,26456,28921
156604,KI270713.1,29478,30798
156605,KI270713.1,31532,33156


## filtering

In [7]:
ad_rna = ad[:, ad.var['feature_types']=='Gene Expression']
ad_atac = ad[:, ad.var['feature_types']=='Peaks']
ad_atac.var['chr'] = peak['chr'].values
ad_atac.var['start'] = peak['start'].values
ad_atac.var['end'] = peak['end'].values

# basic stats
sc.pp.filter_cells(ad_rna, min_genes=0)
sc.pp.filter_genes(ad_rna, min_cells=0)
sc.pp.filter_cells(ad_atac, min_genes=0)
sc.pp.filter_genes(ad_atac, min_cells=0)

# a gene need to be expressed in 5% cells
# a peak need to be accessible in 5% cells
thres = int(ad.shape[0]*0.05)
ad_rna = ad_rna[:, ad_rna.var['n_cells']>thres]
ad_atac = ad_atac[:, ad_atac.var['n_cells']>thres]

<ipython-input-7-4742c4ba5bba>:3: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  ad_atac.var['chr'] = peak['chr'].values
/usr/local/lib/python3.10/dist-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


## Save h5ad

In [8]:
chrs = ['chr'+str(i) for i in range(1,23)] + ['chrX', 'chrY']
ad_atac = ad_atac[:, ad_atac.var['chr'].isin(chrs)]
ad_atac.write('atac_ad.h5ad')

/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1230: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


# Create H5

In [9]:
! wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz

--2023-08-21 06:03:21--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘hg38.fa.gz’

hg38.fa.gz          100%[===================>] 938.09M  41.5MB/s    in 24s     

2023-08-21 06:03:45 (38.4 MB/s) - ‘hg38.fa.gz’ saved [983659424/983659424]



In [10]:
! gunzip hg38.fa.gz

In [11]:
# ! python /content/scBasset/bin/scbasset_preprocess.py --ad_file atac_ad.h5ad --input_fasta hg38.fa
# # # # # # # # # ! python ../../bin/scbasset_train.py --input_folder processed/ --epochs 1000

checking processed files

In [ ]:
# ad_processed = sc.read_10x_h5("/content/processed/train_seqs.h5", gex_only=False)
# ad_processed

AttributeError: ignored

In [ ]:
# m_train = np.load("/content/processed/m_train.npz", {})
# m_train.files

['indices', 'indptr', 'format', 'shape', 'data']

In [ ]:
# Counter(m_train["data"])

# Using BioPython

In [ ]:
next(iter(SeqIO.parse("/content/hg38.fa", "fasta")))

In [ ]:
record_dict = SeqIO.index("/content/hg38.fa", "fasta")
len(record_dict)

In [27]:
Counter(record_dict["chr1"].seq)

Counter({'N': 18475229,
         't': 34645045,
         'a': 34523502,
         'c': 24997418,
         'g': 25044396,
         'C': 23057625,
         'A': 32546775,
         'G': 23067132,
         'T': 32599119,
         'n': 181})